In [1]:
# global constants
g_maxAngVel = pi / 2 # move in azimuth or elevation at max rate of 90 deg / sec
g_maxScanRate = 1000 # scans points at the rate of 1000 pts/sec

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in PCS
# output: nx3 list of points in MCS
function pcs_to_mcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = inv(mcs) * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in MCS
# output: nx3 list of points in PCS
function mcs_to_pcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = mcs * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian to spherical
# input:  pts: nx3 list of xyz (cartesian) points
# output: nx3 list of spherical points (range, theta, phi)
function xyz_to_sph(pts)
    r = [vecnorm(pts[i,:]) for i=1:size(pts,1)]
    r2= [vecnorm(pts[i,1:2]) for i=1:size(pts,1)]
    t = atan2.(r2, pts[:,3])
    p = atan2.(pts[:,2], pts[:,1])
    return [r t p]
end;

# function to convert points from spherical to cartesian
# input:  pts: nx3 list of (r,t,p) (spherical) points
# output: nx3 list of cartesian points (xyz)
function sph_to_xyz(pts)
    # multiply by inverse of the LR's location/orientation
    x = pts[:,1] .* sin.(pts[:,2]) .* cos.(pts[:,3]) # rsinθcosϕ
    y = pts[:,1] .* sin.(pts[:,2]) .* sin.(pts[:,3]) # rsinθsinϕ
    z = pts[:,1] .* cos.(pts[:,2])                   # rcosθ
    return [x y z]
end;


# function to simulate a scan from one spherical coord to another
# input: from: spherical coordinate triple (radius, theta, phi) to start from
#        to:   spherical coordinate triple (radius, theta, phi) to end at
# output: n x 3 list of spherical points
function scan_from_to(from, to)
    delta = to .- from
    time = maximum(abs.(delta[2:3])) / g_maxAngVel
    qty = floor(time * g_maxScanRate)
    rgs = linspace.(from, to, qty)
    return [collect(rgs[1]) collect(rgs[2]) collect(rgs[3])]
end

# function to simulate a scan through a list of (spherical) points
# input:  list of points (spherical) in the order to be visited
# output: list of points (spherical) of the simulated scan points
function scan_tour(pts)
    ret = []
    for i = 1:size(pts,1)-1
        line = scan_from_to(pts[i,:], pts[i+1,:])
        ret = [ret; line]
    end
    return ret
end

function test_transforms()
    # load the feature file
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # convert to spherical
    pts_mcs = pcs_to_mcs_xyz(lr_tfm, pts_pcs) # convert feature locations to MCS
    pts_lr = xyz_to_sph(pts_mcs) # convert feature locations to spherical

    # round trip as a test
    pts_mcs_back = sph_to_xyz(pts_lr)
    pts_pcs_back = mcs_to_pcs_xyz(lr_tfm, pts_mcs_back)

    # points should be the same
    diff = pts_pcs - pts_pcs_back
    display(maximum(diff))
end

function test_scan()
    # load data.
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # shuffle an index vector
    idx = Vector(1:size(pts_pcs,1))
    rng = MersenneTwister(1234)
    idx = shuffle(rng, idx)
    
    # shuffle input pts and convert them to spherical
    sph = xyz_to_sph(pcs_to_mcs_xyz(lr_tfm, pts_pcs[idx,:]))

    # generate scan data
    scan = scan_tour(sph)
    
    # back to pcs
    scan = mcs_to_pcs_xyz(lr_tfm, sph_to_xyz(scan))
    println(size(scan))
    
    # save
    writecsv("scan1.csv", scan)
end

test_transforms()
test_scan()

8.881784197001252e-16

(12146, 3)


In [2]:
data = readcsv("50_Holes_FrontLeft.csv");
pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

# shuffle an index vector
idx = Vector(1:size(pts_pcs,1))
rng = MersenneTwister(1234)
idx = shuffle(rng, idx)

# shuffle input pts and convert them to spherical
sph = xyz_to_sph(pcs_to_mcs_xyz(lr_tfm, pts_pcs[idx,:]));

In [3]:
max_vel = pi/4

0.7853981633974483

In [4]:
sph

51×3 Array{Float64,2}:
 2.62107  1.80213  -2.04955
 2.64078  1.2668   -2.20451
 2.51207  1.73556  -2.43765
 2.07666  1.49795  -2.20861
 2.05449  1.63709  -2.18245
 2.63236  1.27712  -2.20449
 2.40443  1.44661  -2.10753
 2.48488  1.35118  -2.15744
 2.33109  1.27382  -2.26356
 2.61359  1.78966  -2.04956
 2.65872  1.26153  -2.38886
 2.35255  2.08092  -2.46887
 2.12386  1.57002  -2.28567
 ⋮                         
 2.14254  1.59942  -2.29792
 2.55551  2.02866  -2.55462
 2.38126  1.24913  -2.27631
 2.09857  1.42692  -2.20658
 2.10379  1.68632  -2.16959
 3.62157  1.71344  -2.47656
 2.2416   1.74533  -2.13543
 2.55887  1.77337  -2.06038
 2.07729  1.9259   -2.24952
 2.38354  1.74833  -2.38786
 2.08857  1.98017  -2.28272
 2.12791  1.62944  -2.28575

In [5]:
using JuMP, Gurobi
using NamedArrays
m = Model(solver=GurobiSolver(OutputFlag=0))
@variable(m, x[1:51, 1:51], Bin)

51×51 Array{JuMP.Variable,2}:
 x[1,1]   x[1,2]   x[1,3]   x[1,4]   x[1,5]   …  x[1,49]   x[1,50]   x[1,51] 
 x[2,1]   x[2,2]   x[2,3]   x[2,4]   x[2,5]      x[2,49]   x[2,50]   x[2,51] 
 x[3,1]   x[3,2]   x[3,3]   x[3,4]   x[3,5]      x[3,49]   x[3,50]   x[3,51] 
 x[4,1]   x[4,2]   x[4,3]   x[4,4]   x[4,5]      x[4,49]   x[4,50]   x[4,51] 
 x[5,1]   x[5,2]   x[5,3]   x[5,4]   x[5,5]      x[5,49]   x[5,50]   x[5,51] 
 x[6,1]   x[6,2]   x[6,3]   x[6,4]   x[6,5]   …  x[6,49]   x[6,50]   x[6,51] 
 x[7,1]   x[7,2]   x[7,3]   x[7,4]   x[7,5]      x[7,49]   x[7,50]   x[7,51] 
 x[8,1]   x[8,2]   x[8,3]   x[8,4]   x[8,5]      x[8,49]   x[8,50]   x[8,51] 
 x[9,1]   x[9,2]   x[9,3]   x[9,4]   x[9,5]      x[9,49]   x[9,50]   x[9,51] 
 x[10,1]  x[10,2]  x[10,3]  x[10,4]  x[10,5]     x[10,49]  x[10,50]  x[10,51]
 x[11,1]  x[11,2]  x[11,3]  x[11,4]  x[11,5]  …  x[11,49]  x[11,50]  x[11,51]
 x[12,1]  x[12,2]  x[12,3]  x[12,4]  x[12,5]     x[12,49]  x[12,50]  x[12,51]
 x[13,1]  x[13,2]  x[13,3]  x[13,4

In [6]:
sph[1:5,:]

5×3 Array{Float64,2}:
 2.62107  1.80213  -2.04955
 2.64078  1.2668   -2.20451
 2.51207  1.73556  -2.43765
 2.07666  1.49795  -2.20861
 2.05449  1.63709  -2.18245

In [7]:
# each node must be touched
for i in 1:51
    cons = @constraint(m, sum(x[i,j] for j in 1:51) == 1)
end

for i in 1:51
    cons = @constraint(m, sum(x[j,i] for j in 1:51) == 1)
end

# no self loops
for i in 1:51
    cons = @constraint(m, x[i, i] == 0)
end

In [8]:
@objective(m, Min, sum(max_vel*x[i,j]*(sph[i, 2] - sph[j, 2]) + max_vel*x[i,j]*(sph[i, 3] - sph[j, 3]) for i in 1:51, j in 1:51));

In [9]:
solve(m)

Academic license - for non-commercial use only


:Optimal

In [10]:
getobjectivevalue(m)

-1.1102230246251565e-15

In [11]:
x_opt = getvalue(x)

51×51 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [19]:
scanned = []
i = 1
flag = true
while(i<51)
    for j in 1:51
        if(x_opt[i, j] > 0)
            push!(scanned, j)
            i += 1
            continue
        end
    end
    if length(scanned) > length(unique(scanned))
        print("Gotcha!")
        break
    end
end

In [22]:
length(unique(scanned))

50

In [21]:
length(scanned)

50

In [23]:
for j in scanned
    println("Scan feature $j")
end

Scan feature 17
Scan feature 45
Scan feature 30
Scan feature 50
Scan feature 10
Scan feature 21
Scan feature 44
Scan feature 40
Scan feature 27
Scan feature 6
Scan feature 42
Scan feature 1
Scan feature 31
Scan feature 37
Scan feature 41
Scan feature 20
Scan feature 35
Scan feature 16
Scan feature 49
Scan feature 22
Scan feature 4
Scan feature 24
Scan feature 3
Scan feature 47
Scan feature 51
Scan feature 43
Scan feature 26
Scan feature 46
Scan feature 8
Scan feature 5
Scan feature 39
Scan feature 11
Scan feature 29
Scan feature 7
Scan feature 2
Scan feature 12
Scan feature 28
Scan feature 13
Scan feature 19
Scan feature 32
Scan feature 36
Scan feature 25
Scan feature 38
Scan feature 14
Scan feature 23
Scan feature 18
Scan feature 48
Scan feature 9
Scan feature 15
Scan feature 34


In [29]:
ordered_features = zeros(51, 3)
i = 1
for j in scanned
    ordered_features[i, :] = sph[j, :]
    i += 1
end

In [30]:
ordered_features

51×3 Array{Float64,2}:
 2.14786  1.35053  -2.19964
 3.62157  1.71344  -2.47656
 2.30693  1.28495  -2.25594
 2.08857  1.98017  -2.28272
 2.61359  1.78966  -2.04956
 2.34655  2.075    -2.46657
 2.10379  1.68632  -2.16959
 2.14254  1.59942  -2.29792
 2.07644  1.67674  -2.17795
 2.63236  1.27712  -2.20449
 2.38126  1.24913  -2.27631
 2.62107  1.80213  -2.04955
 2.28765  1.28805  -2.24639
 ⋮                         
 2.20346  1.74998  -2.14692
 2.21662  1.7806   -2.1468 
 2.26232  1.68191  -2.1781 
 2.25134  1.30636  -2.23964
 2.61918  2.26286  -2.2184 
 2.16722  1.85336  -2.19442
 2.13695  1.87068  -2.20849
 2.07729  1.9259   -2.24952
 2.33109  1.27382  -2.26356
 2.04919  1.66696  -2.18631
 2.64046  1.66894  -2.24316
 0.0      0.0       0.0    

In [31]:
# shuffle input pts and convert them to spherical

# generate scan data
scan = scan_tour(ordered_features)

# back to pcs
scan = mcs_to_pcs_xyz(lr_tfm, sph_to_xyz(scan))
println(size(scan))

# save
writecsv("kartik_scan1.csv", scan)

(13151, 3)


In [32]:
scan

13151×3 Array{Float64,2}:
 0.767131  1.30497  1.96929
 0.760971  1.30081  1.96737
 0.7548    1.29666  1.96543
 0.748617  1.29252  1.96347
 0.742424  1.28839  1.96149
 0.736219  1.28428  1.95949
 0.730003  1.28018  1.95747
 0.723776  1.2761   1.95542
 0.717538  1.27203  1.95336
 0.711289  1.26797  1.95128
 0.705029  1.26392  1.94917
 0.698759  1.25989  1.94704
 0.692478  1.25588  1.9449 
 ⋮                         
 2.00026   3.0      1.52035
 2.00022   3.0      1.5185 
 2.00018   3.0      1.51665
 2.00014   3.0      1.5148 
 2.00011   3.0      1.51295
 2.00008   3.0      1.5111 
 2.00005   3.0      1.50925
 2.00003   3.0      1.5074 
 2.00002   3.0      1.50555
 2.00001   3.0      1.5037 
 2.0       3.0      1.50185
 2.0       3.0      1.5    